<a href="https://colab.research.google.com/github/welcomeglory/Python/blob/master/weatherANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install tensorflow==2.15.0
import tensorflow as tf
print(tf.__version__)

2.15.0


In [21]:
# 사용자 정의 신경망 모델을 정의
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
# Define the model class
class WeatherANN(tf.keras.Model):
    def __init__(self):
        super(WeatherANN, self).__init__()

        self.hid1 = tf.keras.layers.Dense(32,
                                          activation='relu',
                                          kernel_initializer=self.weight_initialization(),
                                          bias_initializer=self.weight_initialization())
        self.drop1 = tf.keras.layers.Dropout(0.2)

        self.hid2 = tf.keras.layers.Dense(32,
                                          activation='relu',
                                          kernel_initializer=self.weight_initialization(),
                                          bias_initializer=self.weight_initialization())
        self.drop2 = tf.keras.layers.Dropout(0.2)


        self.result = tf.keras.layers.Dense(4,
                                            activation=None,
                                            kernel_initializer=self.weight_initialization(),
                                            bias_initializer=self.weight_initialization())

    def weight_initialization(self):
        return tf.keras.initializers.RandomNormal(mean=0., stddev=1.)

    def call(self, x, training=False):  # Forward pass
        y1 = self.hid1(x)
        d1 = self.drop1(y1, training=training)
        y2 = self.hid2(d1)
        d2 = self.drop2(y2, training=training)
        logits = self.result(d2)
        sy = tf.nn.softmax(logits)
        return logits, sy

In [22]:
# TensorFlow를 사용하여 신경망 모델의 손실 함수, 역전파(백워드 패스), 그리고 정확도 계산을 정의하는 방법
# 손실 함수 정의
@tf.function
def cross_entropy_loss(logits, y):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

# 역전파 및 옵티마이저
learningRate = 0.001
optimizer = tf.optimizers.Adam(learningRate)

@tf.function
def backward(model, x, y):
    with tf.GradientTape() as grad:
        logits, sy = model(x, training=True)
        loss = cross_entropy_loss(logits, y)
    grads = grad.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

# 정확도 계산
@tf.function
def accuracy(predY, y):
    correct_predictions = tf.equal(tf.argmax(predY, axis=1), tf.argmax(y, axis=1))
    acc = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    return acc

In [23]:
# 신경망 모델을 훈련하는 함수 train_learning을 정의
def train_learning(epochs, frequency, trainDataIter, model, XTest, yTest):
    y_loss = []
    for epoch in tqdm(range(epochs)):
        bx, by = next(trainDataIter)
        loss = backward(model, bx, by)
        y_loss.append(loss)
        if epoch % frequency == 0:
            predY, sy = model(bx, training=False)
            acc = accuracy(predY, by)
            print(f"Epoch: {epoch+1} ===> Loss: {loss:.4f}, Accuracy: {acc:.4f}")
    predY, sy = model(XTest, training=False)
    acc = accuracy(predY, yTest)
    print(f"Test Accuracy: {acc:.4f}")
    return y_loss

In [24]:
from google.colab import files
uploaded = files.upload()

Saving weather_classification_data.csv to weather_classification_data (2).csv


In [25]:
# 1. 데이터 로딩 및 전처리
# 데이터 로딩
df = pd.read_csv("weather_classification_data (2).csv")

df.keys()
df['Cloud Cover'].unique()

# 'Cloud Cover' 열의 값들을 숫자로 변환
df['Cloud Cover'] = np.where(df['Cloud Cover']=='partly cloudy', 0,
                              np.where(df['Cloud Cover']=='clear', 1,
                                    np.where(df['Cloud Cover']=='overcast', 2, 3)))

# print(df['Season'].unique())
# 'Season' 열의 값들을 숫자로 변환
df['Season'] = np.where(df['Season']=='Winter', 0,
                             np.where(df['Season']=='Spring', 1,
                                      np.where(df['Season']=='Summer', 2, 3)))

# print(df['Location'].unique())
# 'Location' 열의 값들을 숫자로 변환
df['Location'] = np.where(df['Location']=='inland', 0,
                             np.where(df['Location']=='mountain', 1, 2))

# print(df['Weather Type'].unique())
# 'Weather Type' 열의 값들을 숫자로 변환
df['Weather Type'] = np.where(df['Weather Type']=='Rainy', 0,
                             np.where(df['Weather Type']=='Cloudy', 1,
                                      np.where(df['Weather Type']=='Sunny', 2, 3)))

df.keys()
# 입력 피처와 레이블 설정
X = df[['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)',
       'Cloud Cover', 'Atmospheric Pressure', 'UV Index', 'Season',
       'Visibility (km)', 'Location']].values
y = df[['Weather Type']].values

# 데이터 정규화
X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 레이블을 원-핫 인코딩
y_train = tf.one_hot(y_train.flatten(), 4)
y_test = tf.one_hot(y_test.flatten(), 4)

# TensorFlow 데이터셋 생성
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_data = train_data.shuffle(buffer_size=X_train.shape[0]).batch(300).repeat()
trainDataIter = iter(train_data)

# 훈련 설정
epochs = 10000
frequency = 150

# 모델 정의
model = WeatherANN()

# 모델 훈련
y_loss = train_learning(epochs, frequency, trainDataIter, model, X_test, y_test)


  0%|          | 17/10000 [00:02<16:49,  9.89it/s] 

Epoch: 1 ===> Loss: 16.4397, Accuracy: 0.1633


  2%|▏         | 161/10000 [00:04<02:06, 78.05it/s]

Epoch: 151 ===> Loss: 5.6700, Accuracy: 0.5900


  3%|▎         | 307/10000 [00:06<02:15, 71.66it/s]

...

Epoch: 9751 ===> Loss: 0.3914, Accuracy: 0.8800


 99%|█████████▉| 9919/10000 [00:56<00:00, 139.86it/s]

Epoch: 9901 ===> Loss: 0.4144, Accuracy: 0.8967


100%|██████████| 10000/10000 [00:57<00:00, 173.95it/s]

Test Accuracy: 0.8814


In [26]:
# 모델 저장
path = '/content/drive/MyDrive/세종교육/weatherANN/'
model.save(path + f"weatherAnn_tf_{tf.__version__}_version0")

# 모델 구조 요약
model.summary()

Model: "weather_ann_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             multiple                  352       
                                                                 
 dropout_4 (Dropout)         multiple                  0         
                                                                 
 dense_7 (Dense)             multiple                  1056      
                                                                 
 dropout_5 (Dropout)         multiple                  0         
                                                                 
 dense_8 (Dense)             multiple                  132       
                                                                 
Total params: 1540 (6.02 KB)
Trainable params: 1540 (6.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
# 모델 로드
model1 =tf.keras.models.load_model("/content/drive/MyDrive/세종교육/weatherANN/weatherAnn_tf_2.15.0_version0")

# 테스트 데이터 출력
print(X_test[10:11])

# 예측 수행
predY = model.predict(X_test[10:11])

# 실제 레이블 출력
print(int(tf.argmax(y_test[10:11][0], axis=0)))

# 예측 결과 인덱스 출력
print(predY[1].argmax())

# 예측 결과 출력
predY

[[0.20895522 0.61797753 0.95876289 0.79816514 0.66666667 0.49294645
  0.35714286 0.         0.225      0.        ]]
1/1 [==============================] - 0s 83ms/step
3
3


(array([[ 0.79467523, -0.28663212, -1.0769742 ,  4.365909  ]],
       dtype=float32),
 array([[0.02698803, 0.00915304, 0.00415264, 0.95970625]], dtype=float32))